# Calculating travel time to ports based on select road improvements in Tunisia

Unit of analysis is ADM2 (geoboundaries). Need to summarize the following:
- Monthly Nighttime Lights mean and sum
- TT to ports pre and post implementation of TREATED roads
- Distance to nearest treated road, distance to each labelled treated road
- Percentage Urban

In [1]:
import sys
import os
import rasterio

import pandas as pd
import geopandas as gpd
import skimage.graph as graph

from shapely.geometry import Point

sys.path.insert(0, r"C:\WBG\Work\Code\GOSTrocks\src")
import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.dataMisc as dMisc
import GOSTrocks.ntlMisc as ntlMisc
from GOSTrocks.misc import tPrint

sys.path.append(r"C:\WBG\Work\Code\GOSTnetsraster\src")
import GOSTnetsraster.market_access as ma
import GOSTnetsraster.conversion_tables as speed_tables

%load_ext autoreload
%autoreload 2

In [2]:
# Input parameters
m_crs = 32632  # Need to project data to a metres-based projection

# Define input data
base_folder = "C:/WBG/Work/TUN_Impact/"
landcover_file = os.path.join(base_folder, "DATA", "ESA_Globcover.tif")
# These are the digitized road segments that have been improved
treated_segments_file = os.path.join(
    base_folder, "DATA", "DIME_Roads", "treated_roads.shp"
)
control_segments_file = os.path.join(
    base_folder, "DATA", "DIME_Roads", "control_roads.shp"
)
road_segments_file = os.path.join(base_folder, "DATA", "impacted_osm_roads.gpkg")
osm_roads_file = os.path.join(base_folder, "DATA", "OSM", "gis_osm_roads_free_1.shp")
# WorldPop 2020 constrained, projected to m_crs
pop_file = os.path.join(base_folder, "DATA", "tun_ppp_2020_constrained_proj.tif")
# https://datacatalog.worldbank.org/int/search/dataset/0038118/Global---International-Ports
port_file = os.path.join(base_folder, "DATA", "TUN_ports.gpkg")
# administrative boundaries are used to summarize population
tun_adm2 = dMisc.get_geoboundaries("TUN", "ADM2")
tun_adm1 = dMisc.get_geoboundaries("TUN", "ADM1")

# Define output files
friction_folder = os.path.join(base_folder, "DATA", "FRICTION")
results_folder = os.path.join(base_folder, "RESULTS")
for cFolder in [friction_folder, results_folder]:
    if not os.path.exists(cFolder):
        os.makedirs(cFolder)
pre_friction_file = os.path.join(friction_folder, "FRICTION_pre_intervention.tif")
post_friction_file = os.path.join(friction_folder, "FRICTION_post_intervention.tif")
# This extracts the existing global friction file, used only for comparison
global_friction_file = os.path.join(friction_folder, "2020_motorized_friction.geotiff")
if not os.path.exists(global_friction_file):
    gl_fr = rasterio.open(
        r"J:\Data\GLOBAL\INFRA\FRICTION_2020\2020_motorized_friction_surface.geotiff"
    )
    local_fr = rMisc.clipRaster(gl_fr, tun_adm2, global_friction_file)

# Read in data
dests = gpd.read_file(port_file).to_crs(m_crs)
if not os.path.exists(landcover_file):
    global_landcover = (
        r"R:\GLOBAL\LCVR\Globcover\2015\ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7.tif"
    )
    in_lc = rasterio.open(global_landcover)
    temp_landcover_file = landcover_file.replace(".tif", "_temp.tif")
    local_lc = rMisc.clipRaster(in_lc, tun_adm2, temp_landcover_file)
    temp_lc = rasterio.open(temp_landcover_file)
    proj_res = rMisc.project_raster(temp_lc, m_crs)
    with rasterio.open(landcover_file, "w", **proj_res[1]) as outR:
        outR.write(proj_res[0])

in_lc = rasterio.open(landcover_file)
in_pop = rasterio.open(pop_file)
if in_pop.crs != in_lc.crs:
    proj_res = rMisc.standardizeInputRasters(
        in_pop, in_lc, pop_file.replace(".tif", "_proj.tif")
    )

In [ ]:
# Download worldcover data
tiles_geojson = r"C:\WBG\Work\data\LCVR\esa_worldcover_grid.geojson"
in_tiles = gpd.read_file(tiles_geojson)
sel_tiles = in_tiles.loc[in_tiles.intersects(tun_adm2.unary_union)]

tile_path = (
    "s3://esa-worldcover/v200/2021/map/ESA_WorldCover_10m_2021_v200_{tile}_Map.tif"
)
out_folder = os.path.join(base_folder, "DATA", "WorldCover")
for idx, row in sel_tiles.iterrows():
    cur_tile_path = tile_path.format(tile=row["ll_tile"])
    cur_out = os.path.join(out_folder, f"WorldCover_{row['ll_tile']}.tif")
    if not os.path.exists(cur_out):
        command = (
            f"aws s3 --no-sign-request --no-verify-ssl cp {cur_tile_path} {cur_out}"
        )
        print(command)

In [ ]:
# Download and process industrial zones
import urllib.request
import json

with urllib.request.urlopen("https://afi.e-industrie.gov.tn/apps-lots.php") as url:
    industry_zones = json.load(url)
zones_df = pd.DataFrame(industry_zones)

zones_geoms = [Point(x) for x in zip(zones_df["ZONE_LONG"], zones_df["ZONE_LAT"])]
zones_df = gpd.GeoDataFrame(zones_df, geometry=zones_geoms, crs=4326)
zones_df = zones_df.to_crs(m_crs)
zones_df.to_file(
    os.path.join(base_folder, "DATA", "MAPPING", "industrial_zones.gpkg"), driver="GPKG"
)

In [ ]:
# Process roads to create pre and post friction surfaces
sel_roads = gpd.read_file(road_segments_file)
sel_roads = sel_roads.to_crs(m_crs)
sel_roads["speed"] = 10
all_roads = gpd.read_file(osm_roads_file)
all_roads = all_roads.to_crs(m_crs)
all_roads["speed"] = all_roads["fclass"].map(speed_tables.osm_speed_dict)
all_roads["speed"] = all_roads["speed"].fillna(10.0)
wb_roads_ids = sel_roads.loc[~sel_roads["osm_id"].isna(), "osm_id"]
new_roads = sel_roads.loc[sel_roads["osm_id"].isna(),]

lc_speed_table = speed_tables.esaacci_landcover

In [ ]:
# Generate pre-intervention friction surface
if not os.path.exists(pre_friction_file):
    pre_roads = all_roads.copy()
    pre_roads.loc[pre_roads["osm_id"].isin(wb_roads_ids), "speed"] = 10.0

    pre_friction = ma.generate_roads_lc_friction(
        in_lc,
        pre_roads,
        lc_travel_table=lc_speed_table,
        out_file=pre_friction_file,
        resolution=in_lc.res[0],
    )

pre_friction = rasterio.open(pre_friction_file)

In [ ]:
# Generate post-intervention friction surface
if not os.path.exists(post_friction_file):
    post_roads = all_roads.copy()
    post_roads.loc[post_roads["osm_id"].isin(wb_roads_ids), "speed"] = 40.0

    post_friction = ma.generate_roads_lc_friction(
        in_lc,
        post_roads,
        lc_travel_table=lc_speed_table,
        out_file=post_friction_file,
        resolution=in_lc.res[0],
    )

post_friction = rasterio.open(post_friction_file)

# Calculate travel time

In [ ]:
# Calculate pre-intervention, population-weighted travel times summarized at admin 2
frictionD = pre_friction.read()[0, :, :]
frictionD = frictionD * pre_friction.res[0]
mcp = graph.MCP_Geometric(frictionD)
pre_tt_ports = ma.summarize_travel_time_populations(
    in_pop, pre_friction, dests, mcp, tun_adm2
)
pd.DataFrame(pre_tt_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "PRE_ADM2_tt_ports.csv")
)

pre_zones_ports = ma.summarize_travel_time_populations(
    in_pop, pre_friction, zones_df, mcp, tun_adm2
)
pd.DataFrame(pre_zones_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "PRE_ADM2_tt_zones.csv")
)

In [ ]:
# Calculate pre-intervention, population-weighted travel times summarized at admin 1
frictionD = pre_friction.read()[0, :, :]
frictionD = frictionD * pre_friction.res[0]
mcp = graph.MCP_Geometric(frictionD)
pre_tt_ports = ma.summarize_travel_time_populations(
    in_pop, pre_friction, dests, mcp, tun_adm1
)
pd.DataFrame(pre_tt_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "PRE_ADM1_tt_ports.csv")
)

pre_zones_ports = ma.summarize_travel_time_populations(
    in_pop, pre_friction, zones_df, mcp, tun_adm1
)
pd.DataFrame(pre_zones_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "PRE_ADM1_tt_zones.csv")
)

In [ ]:
# Calculate pre-intervention, population-weighted travel time
frictionD = post_friction.read()[0, :, :]
frictionD = frictionD * post_friction.res[0]
mcp = graph.MCP_Geometric(frictionD)
post_tt_ports = ma.summarize_travel_time_populations(
    in_pop, post_friction, dests, mcp, tun_adm2
)
pd.DataFrame(post_tt_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "POST_ADM2_tt_ports.csv")
)

post_zones_ports = ma.summarize_travel_time_populations(
    in_pop, post_friction, zones_df, mcp, tun_adm2
)
pd.DataFrame(post_zones_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "POST_ADM2_tt_zones.csv")
)

In [ ]:
# Calculate pre-intervention, population-weighted travel time
frictionD = post_friction.read()[0, :, :]
frictionD = frictionD * post_friction.res[0]
mcp = graph.MCP_Geometric(frictionD)
post_tt_ports = ma.summarize_travel_time_populations(
    in_pop, post_friction, dests, mcp, tun_adm1
)
pd.DataFrame(post_tt_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "POST_ADM1_tt_ports.csv")
)

post_zones_ports = ma.summarize_travel_time_populations(
    in_pop, post_friction, zones_df, mcp, tun_adm1
)
pd.DataFrame(post_zones_ports.drop(["geometry"], axis=1)).to_csv(
    os.path.join(results_folder, "POST_ADM1_tt_zones.csv")
)

# Zonal stats on nighttimelights

In [3]:
ntl_files = ntlMisc.aws_search_ntl()

c:\wbg\Anaconda3\envs\gn\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'globalnightlight.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
# Clip out NTL raster files for pre and post implementation
pre_date = "201501"
post_date = "202401"
pre_file = [x for x in ntl_files if "201501" in x][0]
post_file = [x for x in ntl_files if "202401" in x][0]

ntl_out_folder = os.path.join(base_folder, "DATA", "NTL_Rasters")
with rasterio.Env(GDAL_HTTP_UNSAFESSL="YES") as env:
    pre_res = rMisc.clipRaster(
        rasterio.open(pre_file),
        tun_adm1,
        os.path.join(ntl_out_folder, "VIIRS_201501.tif"),
    )
    post_res = rMisc.clipRaster(
        rasterio.open(post_file),
        tun_adm1,
        os.path.join(ntl_out_folder, "VIIRS_202401.tif"),
    )

In [7]:
ntl_diff = post_res[0] - pre_res[0]
with rasterio.open(
    os.path.join(ntl_out_folder, "VIIRS_201501_202401.tif"), "w", **pre_res[1]
) as outR:
    outR.write(ntl_diff)

# Measure distance to treated roads

In [ ]:
tPrint("Start")
for road_file, out_file, col_lbl in [
    [treated_segments_file, "distance_to_treated_roads.csv", "RTE_NOM"],
    [control_segments_file, "distance_to_control_roads.csv", "RTE_NOM"],
    [road_segments_file, "distance_to_WB_digitized_roads.csv", "road_group"],
]:
    roads = gpd.read_file(road_file)
    roads = roads.to_crs(m_crs)
    tun_adm2 = tun_adm2.to_crs(m_crs)
    for lbl, df in roads.groupby(col_lbl):
        tun_adm2[f"dist_{lbl}"] = tun_adm2.apply(
            lambda x: x["geometry"].distance(df.union_all()), axis=1
        )
    tun_adm2["dist_road"] = tun_adm2.apply(
        lambda x: x["geometry"].distance(roads.union_all()), axis=1
    )
    pd.DataFrame(tun_adm2.drop(["geometry"], axis=1)).to_csv(
        os.path.join(base_folder, "RESULTS", out_file)
    )
    tPrint(out_file)

In [ ]:
gpd.read_file(control_segments_file)["RTE_NOM"].unique()

# PRepare mapping data

In [3]:
results_csv = os.path.join(base_folder, "DATA", "MAPPING", "mapping_res.csv")
in_res = pd.read_csv(results_csv)

In [4]:
pd.merge(tun_adm1, in_res, left_on="shapeName", right_on="adm1", how="outer").to_file(
    os.path.join(base_folder, "DATA", "MAPPING", "adm1_res.gpkg"), driver="GPKG"
)

In [5]:
pd.merge(
    tun_adm1, in_res, left_on="shapeName", right_on="adm1", how="outer"
).sort_values("time_hub")

,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry,adm1,time_hub,time_port,per_pov,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Ariana,TN-12,13205935B58390004509121,TUN,ADM1,"POLYGON ((10.25216 36.94316, 10.2509 36.94384,...",Ariana,0.000000,0.000000,5.4,NaN,NaN,NaN,NaN
2,Bizerte,TN-23,13205935B29646166511918,TUN,ADM1,"MULTIPOLYGON (((8.89859 37.51906, 8.89867 37.5...",Bizerte,0.000000,0.000000,17.5,NaN,NaN,NaN,NaN
3,Béja,TN-31,13205935B67114336122672,TUN,ADM1,"POLYGON ((9.01615 36.46342, 9.01665 36.46257, ...",Béja,0.000000,0.000000,32.0,NaN,NaN,NaN,NaN
5,Gabès,TN-81,13205935B69181748376292,TUN,ADM1,"POLYGON ((9.7111 34.2815, 9.70066 34.28223, 9....",Gabès,0.000000,0.000000,15.8,NaN,NaN,NaN,NaN
7,Jendouba,TN-32,13205935B57742642676849,TUN,ADM1,"POLYGON ((8.93967 36.45343, 8.94174 36.45326, ...",Jendouba,0.000000,0.000000,22.4,NaN,NaN,NaN,NaN
6,Gafsa,TN-71,13205935B54080015312342,TUN,ADM1,"POLYGON ((8.32421 34.73281, 8.32003 34.73109, ...",Gafsa,0.000000,0.000000,18.0,NaN,NaN,NaN,NaN
10,Kébili,TN-73,13205935B11721331776240,TUN,ADM1,"POLYGON ((7.73658 33.42466, 7.73256 33.3747, 7...",Kébili,0.000000,0.000000,18.5,NaN,NaN,NaN,NaN
9,Kasserine,TN-42,13205935B52637504718586,TUN,ADM1,"POLYGON ((8.35396 35.61661, 8.35295 35.61567, ...",Kasserine,0.000000,0.030598,32.8,NaN,NaN,NaN,NaN
14,Médenine,TN-82,13205935B11392830158982,TUN,ADM1,"MULTIPOLYGON (((10.86005 33.59917, 10.86158 33...",Médenine,0.000000,0.000000,21.6,NaN,NaN,NaN,NaN
12,Manouba,TN-14,13205935B47286197858453,TUN,ADM1,"POLYGON ((10.11496 36.82753, 10.11442 36.82679...",Manouba,0.000000,0.000000,12.1,NaN,NaN,NaN,NaN
